In [6]:
!pip install -r ../../../requirements.txt

  Using cached earthengine_api-1.5.4-py3-none-any.whl.metadata (2.1 kB)
  Using cached google_api_python_client-2.162.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_api_core-2.24.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_cloud_core-2.4.2-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.6.0-cp39-cp39-macosx_12_0_x86_64.whl.metadata (2.3 kB)
  Using cached googleapis_common_protos-1.68.0-py2.py3-none-any.whl.metadata (5.1 kB)
  Using cache

In [7]:
import ee
import xarray as xr

In [8]:
ee.Authenticate()
ee.Initialize()

In [9]:
image_asset = "USGS/SRTMGL1_003"
image = ee.Image(image_asset)

In [10]:
sea_to_sky_extent = (-123.5, 49.5, -122.5, 50.5)
sea_to_sky = ee.Geometry.Rectangle(sea_to_sky_extent)

In [13]:
scale = 30
crs = "EPSG:4326"
out_path = "test.tif"

In [19]:
ix = xr.open_dataset(
    ee.ImageCollection(image),
    engine="ee",
    scale=scale,
    projection=crs,
    geometry=sea_to_sky,
)
ix

/Applications/QGIS-LTR.app/Contents/MacOS/lib/python3.9/site-packages/xee/ext.py:688: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


<xarray.Dataset>
Dimensions:    (time: 1, lon: 1, lat: 1)
Coordinates:
  * time       (time) int64 0
  * lon        (lon) float64 -108.5
  * lat        (lat) float64 64.5
Data variables:
    elevation  (time, lon, lat) float32 ...
Attributes:
    crs:      EPSG:4326

In [ ]:
# issue with mismatched proj...
!export PROJ_LIB=/Applications/QGIS-LTR.app/Contents/Resources/proj

In [22]:
ix.rio.to_raster(out_path)

MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this.

In [ ]:
ix_rename = ix.rename({"lon": "x", "lat": "y"})
print(ix_rename)
ix_rename.rio.to_raster(out_path)

ERROR 1: PROJ: internal_proj_create_from_database: /Applications/QGIS-LTR.app/Contents/Resources/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 3 is expected. It comes from another PROJ installation.


TooManyDimensions: Only 2D and 3D data arrays supported.

In [25]:
ix_spatial_dims = ix_rename.rio.set_spatial_dims(x_dim="x", y_dim="y")
print(ix_spatial_dims)
ix_spatial_dims.rio.to_raster(out_path)

<xarray.Dataset>
Dimensions:    (time: 1, x: 1, y: 1)
Coordinates:
  * time       (time) int64 0
  * x          (x) float64 -108.5
  * y          (y) float64 64.5
Data variables:
    elevation  (time, x, y) float32 nan
Attributes:
    crs:      EPSG:4326


TooManyDimensions: Only 2D and 3D data arrays supported.

In [27]:
ix_drop_time = ix_spatial_dims.isel(time=0, drop=True)
print(ix_drop_time)
ix_drop_time.rio.to_raster(out_path)

<xarray.Dataset>
Dimensions:    (x: 1, y: 1)
Coordinates:
  * x          (x) float64 -108.5
  * y          (y) float64 64.5
Data variables:
    elevation  (x, y) float32 nan
Attributes:
    crs:      EPSG:4326


ERROR 1: PROJ: internal_proj_create_from_database: /Applications/QGIS-LTR.app/Contents/Resources/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 3 is expected. It comes from another PROJ installation.


InvalidDimensionOrder: Invalid dimension order. Expected order: ('band_db068e11-4eb4-47b6-894f-dbe516dbe350', 'y', 'x'). You can use `DataArray.transpose('band_db068e11-4eb4-47b6-894f-dbe516dbe350', 'y', 'x')` to reorder your dimensions.

In [28]:
data_var = list(ix.data_vars.keys())[0]  # Get the first variable name
ix_data_var = ix_drop_time[data_var]
print(ix_data_var)
ix_data_var.rio.to_raster(out_path)

<xarray.DataArray 'elevation' (x: 1, y: 1)>
array([[nan]], dtype=float32)
Coordinates:
  * x        (x) float64 -108.5
  * y        (y) float64 64.5
Attributes:
    id:             elevation
    data_type:      {'type': 'PixelType', 'precision': 'int', 'min': -32768, ...
    dimensions:     [1296001, 417601]
    crs:            EPSG:4326
    crs_transform:  [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002...


InvalidDimensionOrder: Invalid dimension order. Expected order: ('y', 'x'). You can use `DataArray.transpose('y', 'x')` to reorder your dimensions. Data variable: elevation

In [29]:
ix_transpose = ix_data_var.transpose("y", "x")
print(ix_transpose)
ix_transpose.rio.to_raster(out_path)

<xarray.DataArray 'elevation' (y: 1, x: 1)>
array([[nan]], dtype=float32)
Coordinates:
  * x        (x) float64 -108.5
  * y        (y) float64 64.5
Attributes:
    id:             elevation
    data_type:      {'type': 'PixelType', 'precision': 'int', 'min': -32768, ...
    dimensions:     [1296001, 417601]
    crs:            EPSG:4326
    crs_transform:  [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002...


ERROR 1: PROJ: internal_proj_create_from_database: /Applications/QGIS-LTR.app/Contents/Resources/proj/proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 1 whereas a number >= 3 is expected. It comes from another PROJ installation.
/Applications/QGIS-LTR.app/Contents/MacOS/lib/python3.9/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


In [31]:
import rioxarray as rio

ds = rio.open_rasterio(out_path)
ds

<xarray.DataArray (band: 1, y: 1, x: 1)>
[1 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 0.5
  * y            (y) float64 0.5
    spatial_ref  int64 0
Attributes:
    crs_transform:  [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002...
    data_type:      {'type': 'PixelType', 'precision': 'int', 'min': -32768, ...
    dimensions:     [1296001, 417601]
    id:             elevation
    AREA_OR_POINT:  Area
    scale_factor:   30.0
    add_offset:     0.0
    long_name:      elevation